# Gretel AI to Opik Dataset Integration - Complete Cookbook

A comprehensive guide with ready-to-run examples for generating synthetic Q&A datasets using Gretel Navigator and importing them into Opik for model evaluation.

---

## 🎯 What This Cookbook Covers

- **Authentication setup** for both Gretel and Opik
- **Synthetic data generation** using Gretel Navigator
- **Data format conversion** from Gretel to Opik
- **Dataset import** into Opik for evaluation
- **Complete examples** for different use cases

---

## 📋 Prerequisites & Setup

Before starting, you'll need:
1. **Gretel Account**: Sign up at [gretel.ai](https://gretel.ai)
2. **Comet Account**: Sign up at [comet.com](https://comet.com) for Opik access
3. **API Keys**: Gretel API key and Comet API key

### Install Required Packages

In [1]:
%pip install gretel_client opik langchain tiktoken pandas --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass
from gretel_client import Gretel
import opik
import pandas as pd
import json

print("🚀 Starting Gretel to Opik integration setup...")

#set up Opik
opik.configure()

# Set up Gretel API key
if "GRETEL_API_KEY" not in os.environ:
    os.environ["GRETEL_API_KEY"] = getpass.getpass("Enter your Gretel API key: ")

gretel = Gretel(api_key=os.environ["GRETEL_API_KEY"], cache=True, validate=True)

OPIK: Opik is already configured. You can check the settings by viewing the config file at /home/mavrick/.opik.config


🚀 Starting Gretel to Opik integration setup...
Logged in as mavrickrishi@gmail.com ✅


## Find Working Model

In [3]:
def find_working_navigator_model():
    """Find a Navigator model that works in the current environment with enhanced error handling"""
    models_to_try = [
        'gretelai/auto', 
        'gretelai/apache-2.0', 
        'gretelai/llama-3.x',
        'gretelai/gpt-3.5-turbo'
    ]
    
    print(f"🔍 Testing {len(models_to_try)} available models...")
    
    for i, model in enumerate(models_to_try, 1):
        try:
            print(f"🔄 Testing model {i}/{len(models_to_try)}: {model}")
            
            # Initialize navigator with timeout handling
            test_navigator = gretel.factories.initialize_navigator_api(
                "tabular", 
                backend_model=model
            )
            
            print(f"Backend model: {model}")
            print("Navigator Tabular initialized 🚀")
            
            # More comprehensive test with timeout
            print("🧪 Running test generation...")
            test_result = test_navigator.generate(
                "Create a small dataset with columns 'name' and 'age' for 3 people.", 
                num_records=3
            )
            
            # Validate result structure
            if test_result is not None and len(test_result) > 0:
                # Check if expected columns exist
                if 'name' in test_result.columns and 'age' in test_result.columns:
                    print(f"✅ Model {model} works! Generated {len(test_result)} records")
                    print(f"📊 Test database:")
                    print(test_result)
                    print(f"📋 Columns: {list(test_result.columns)}")
                    print(f"📈 Data types: {dict(test_result.dtypes)}")
                    return test_navigator, model
                else:
                    print(f"⚠️ Model {model} returned data but with unexpected columns: {list(test_result.columns)}")
                    # Still might work for our use case, continue testing
                    if len(test_result.columns) >= 2:  # At least some structured data
                        print(f"✅ Model {model} works with alternative structure! Generated {len(test_result)} records")
                        print(f"📊 Test database:")
                        print(test_result)
                        return test_navigator, model
            else:
                print(f"⚠️ Model {model} returned empty or invalid data")
                
        except Exception as e:
            error_msg = str(e)
            print(f"❌ Model {model} failed: {error_msg[:100]}{'...' if len(error_msg) > 100 else ''}")
            
            # Check for specific error types
            if "authentication" in error_msg.lower():
                print("🔐 Authentication issue detected - check your API key")
            elif "timeout" in error_msg.lower():
                print("⏱️ Timeout issue - network or service may be slow")
            elif "quota" in error_msg.lower() or "limit" in error_msg.lower():
                print("📊 Rate limit or quota issue detected")
    
    # If we get here, no model worked
    print("\n❌ No working Navigator model found!")
    print("🔧 Troubleshooting suggestions:")
    print("   1. Check your Gretel API key is valid")
    print("   2. Verify your internet connection")
    print("   3. Check if your account has sufficient credits")
    print("   4. Try again in a few minutes (rate limiting)")
    
    raise Exception("No working Navigator model found. Check your API key and network connection.")

def validate_navigator_setup():
    """Validate that navigator is properly set up"""
    required_vars = ['gretel']
    missing_vars = []
    
    for var_name in required_vars:
        if var_name not in globals() or globals()[var_name] is None:
            missing_vars.append(var_name)
    
    if missing_vars:
        raise NameError(f"Required variables not found: {missing_vars}. Please run setup cells first.")
    
    return True

# Main execution
try:
    print("🔍 Validating setup...")
    validate_navigator_setup()
    
    print("🔍 Finding working Navigator model...")
    navigator, working_model = find_working_navigator_model()
    
    print(f"\n🎉 Setup completed successfully!")
    print(f"✅ Using working model: {working_model}")
    print(f"📊 Navigator type: {type(navigator).__name__}")
    
    # Verify navigator is available globally
    if navigator is None:
        raise Exception("Navigator initialization failed - navigator is None")
    
    # Store in global scope for other cells
    globals()['navigator'] = navigator
    globals()['working_model'] = working_model
    
    print("✅ Navigator and working_model are now available globally")
    
except Exception as e:
    print(f"\n❌ Setup failed: {e}")
    print("\n🔧 Please check:")
    print("   1. Run Cell 1 (install packages) first")
    print("   2. Run Cell 2 (authentication setup) first") 
    print("   3. Check your Gretel API key")
    print("   4. Verify internet connection")
    
    # Set fallback values to prevent downstream errors
    navigator = None
    working_model = None
    
    raise

🔍 Validating setup...
🔍 Finding working Navigator model...
🔍 Testing 4 available models...
🔄 Testing model 1/4: gretelai/auto
Backend model: gretelai/auto
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀
Backend model: gretelai/auto
Navigator Tabular initialized 🚀
🧪 Running test generation...


Generating records:   0%|          | 0/3 [00:00,? records/s]

Received error during generation: 'Unfortunately the AI system could not return data. Please make sure that your request is valid and relevant for tabular data, and does not include sensitive topics. Clearly writing out your request with examples or bulleted list of columns can help. (error_code: MalformedResponseError) Please try again or contact support.'. Retrying.


Generating records: 100%|██████████| 3/3 [00:20, 0.15 records/s]

✅ Model gretelai/auto works! Generated 3 records
📊 Test database:
            name  age
0    Aisha Patel   25
1      Liam Chen   30
2  Nalani Jensen   28
📋 Columns: ['name', 'age']
📈 Data types: {'name': dtype('O'), 'age': dtype('int64')}

🎉 Setup completed successfully!
✅ Using working model: gretelai/auto
📊 Navigator type: TabularInferenceAPI
✅ Navigator and working_model are now available globally


## 📝 Configure Prompt and Source Content

In [4]:
# Base prompt for Q&A dataset creation
PROMPT = (
    "From the source text below, create a dataset with the following columns:\n"
    "* `question`: Ask a set of unique questions related to the topic that a customer might ask. "
    "Questions should be relatively complex and specific enough to be addressed in a short answer.\n"
    "* `context`: Copy the exact sentence(s) from the source text and surrounding details from where the answer can be derived.\n"
    "* `truth`: Respond to the question with a clear, textbook quality answer that provides relevant details to fully address the question.\n"
)

# Your source content (customize this with your domain-specific content)
source_text = """
Artificial Intelligence (AI) has revolutionized numerous industries by automating complex tasks 
and providing intelligent insights. Machine learning, a subset of AI, enables systems to learn 
from data without explicit programming. Deep learning, using neural networks with multiple layers, 
has achieved breakthroughs in image recognition, natural language processing, and decision making.
The field continues to evolve with advancements in transformer architectures, reinforcement learning,
and federated learning approaches that preserve privacy while enabling collaborative model training.
"""

print("📝 Prompt and source text configured")
print(f"📄 Source text length: {len(source_text)} characters")

📝 Prompt and source text configured
📄 Source text length: 596 characters


## 🚀 Generate Synthetic Q&A Dataset

In [5]:
def generate_qa_dataset_robust(navigator, prompt, source_text, max_attempts=3):
    """Generate Q&A dataset with multiple fallback strategies"""
    
    # Different parameter strategies (from complex to simple)
    strategies = [
        {"num_records": 10, "temperature": 0.7, "top_p": 0.9},
        {"num_records": 8, "temperature": 0.5},
        {"num_records": 5},
        {"num_records": 3, "temperature": 0.3}
    ]
    
    for attempt, params in enumerate(strategies, 1):
        try:
            print(f"🔄 Attempt {attempt}: Generating with params {params}")
            result = navigator.generate(f"{prompt}\n\n{source_text}", **params)
            
            if len(result) > 0:
                print(f"✅ Success! Generated {len(result)} records")
                return result
            else:
                print("⚠️ Empty result, trying next strategy...")
                
        except Exception as e:
            print(f"❌ Attempt {attempt} failed: {e}")
            if attempt < len(strategies):
                print("🔄 Trying next strategy...")
    
    return None

# Generate the dataset
print("🚀 Starting data generation...")
synthetic_df = generate_qa_dataset_robust(navigator, PROMPT, source_text)

if synthetic_df is not None:
    print(f"\n📊 Dataset generated successfully!")
    print(f"   Shape: {synthetic_df.shape}")
    print(f"   Columns: {list(synthetic_df.columns)}")
    
    # Display sample data
    pd.set_option('display.max_colwidth', 100)
    print(f"\n📋 Sample generated data:")
    print(synthetic_df.head(3))
else:
    print("❌ Failed to generate any data")

🚀 Starting data generation...
🔄 Attempt 1: Generating with params {'num_records': 10, 'temperature': 0.7, 'top_p': 0.9}


Generating records: 100%|██████████| 10/10 [00:28, 0.35 records/s]

✅ Success! Generated 10 records

📊 Dataset generated successfully!
   Shape: (10, 3)
   Columns: ['question', 'context', 'truth']

📋 Sample generated data:
                                                  question  \
0  What is the primary function of machine learning in AI?   
1         What are some key applications of deep learning?   
2               What is the purpose of federated learning?   

                                                                                               context  \
0   Machine learning, a subset of AI, enables systems to learn from data without explicit programming.   
1  Deep learning, using neural networks with multiple layers, has achieved breakthroughs in image r...   
2  The field continues to evolve with advancements in transformer architectures, reinforcement lear...   

                                                                                                 truth  
0  Machine learning allows systems to learn from data without exp

## 🔄 Convert Gretel Format to Opik Format

In [6]:
# Cell 6 - Convert Gretel Format to Opik Format

def convert_gretel_to_opik_format(df, model_name="unknown"):
    """Enhanced conversion with better validation and error handling"""
    
    if df is None or df.empty:
        raise ValueError("Input DataFrame is None or empty")
    
    print(f"🔄 Converting {len(df)} rows to Opik format...")
    print(f"📝 Available columns: {list(df.columns)}")
    
    # Enhanced column detection
    column_mapping = {
        'question': ['question', 'query', 'q', 'prompt'],
        'answer': ['truth', 'answer', 'response', 'reply', 'a'],
        'context': ['context', 'background', 'source', 'passage']
    }
    
    detected_columns = {}
    for opik_col, possible_names in column_mapping.items():
        for col in df.columns:
            if any(name in col.lower().strip() for name in possible_names):
                detected_columns[opik_col] = col
                print(f"✅ {opik_col.title()} column: {col}")
                break
    
    # Validate required columns
    if 'question' not in detected_columns:
        raise ValueError("No question column found")
    if 'answer' not in detected_columns:
        raise ValueError("No answer column found")
    
    opik_items = []
    skipped_count = 0
    
    for idx, row in df.iterrows():
        try:
            # Build input with validation
            input_data = {}
            
            question = str(row[detected_columns['question']]).strip() if pd.notna(row.get(detected_columns['question'])) else ""
            if not question or len(question) < 5:
                skipped_count += 1
                continue
            input_data["question"] = question
            
            if 'context' in detected_columns and pd.notna(row.get(detected_columns['context'])):
                context = str(row[detected_columns['context']]).strip()
                if context:
                    input_data["context"] = context
            
            # Get expected output with validation
            answer = str(row[detected_columns['answer']]).strip() if pd.notna(row.get(detected_columns['answer'])) else ""
            if not answer or len(answer) < 10:
                skipped_count += 1
                continue
            
            # Create Opik item
            item = {
                "input": input_data,
                "expected_output": answer,
                "metadata": {
                    "source": "gretel_navigator",
                    "generated": True,
                    "row_index": idx,
                    "model": model_name,
                    "question_length": len(question),
                    "answer_length": len(answer),
                    "has_context": "context" in input_data
                }
            }
            opik_items.append(item)
            
        except Exception as e:
            print(f"❌ Error converting row {idx}: {e}")
            skipped_count += 1
    
    print(f"✅ Successfully converted {len(opik_items)}/{len(df)} rows")
    if skipped_count > 0:
        print(f"⚠️ Skipped {skipped_count} rows due to validation issues")
    
    return opik_items

# Execute the conversion
print("🔄 Starting format conversion...")

# Check if synthetic_df exists and has data
if 'synthetic_df' in locals() and synthetic_df is not None and len(synthetic_df) > 0:
    try:
        # Convert the dataset
        opik_formatted_data = convert_gretel_to_opik_format(synthetic_df, working_model)
        
        if opik_formatted_data:
            print(f"\n📋 Conversion completed successfully!")
            print(f"📊 Converted {len(opik_formatted_data)} items")
            print(f"\n📋 Sample converted item:")
            print(json.dumps(opik_formatted_data[0], indent=2))
            
            # Store in global scope for next cells
            globals()['opik_formatted_data'] = opik_formatted_data
            print(f"\n✅ Converted data is now available globally")
            
        else:
            print("❌ No items were successfully converted")
            globals()['opik_formatted_data'] = None
            
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        print("\n🔧 Troubleshooting:")
        print("   1. Check if synthetic_df has the expected columns")
        print("   2. Verify data quality (non-empty questions/answers)")
        print("   3. Re-run data generation if needed")
        globals()['opik_formatted_data'] = None
        
else:
    print("❌ No synthetic data available to convert")
    print("🔄 Available variables:")
    
    # Debug information
    debug_vars = ['synthetic_df', 'navigator', 'working_model']
    for var in debug_vars:
        if var in locals():
            val = locals()[var]
            print(f"   ✅ {var}: {type(val).__name__} ({len(val) if hasattr(val, '__len__') else 'N/A'})")
        elif var in globals():
            val = globals()[var]
            print(f"   ✅ {var}: {type(val).__name__} (global)")
        else:
            print(f"   ❌ {var}: Not found")
    
    print("\n📋 Please ensure:")
    print("   1. Cell 3 (find working model) completed successfully")
    print("   2. Cell 5 (data generation) completed successfully")
    print("   3. synthetic_df variable contains valid data")
    
    globals()['opik_formatted_data'] = None

print(f"\n🎯 Conversion step completed!")
print(f"Next: Run Cell 7 to push data to Opik")

🔄 Starting format conversion...
🔄 Converting 10 rows to Opik format...
📝 Available columns: ['question', 'context', 'truth']
✅ Question column: question
✅ Answer column: truth
✅ Context column: context
✅ Successfully converted 10/10 rows

📋 Conversion completed successfully!
📊 Converted 10 items

📋 Sample converted item:
{
  "input": {
    "question": "What is the primary function of machine learning in AI?",
    "context": "Machine learning, a subset of AI, enables systems to learn from data without explicit programming."
  },
  "expected_output": "Machine learning allows systems to learn from data without explicit programming, enabling them to improve their performance on tasks over time.",
  "metadata": {
    "source": "gretel_navigator",
    "generated": true,
    "row_index": 0,
    "model": "gretelai/auto",
    "question_length": 55,
    "answer_length": 143,
    "has_context": true
  }
}

✅ Converted data is now available globally

🎯 Conversion step completed!
Next: Run Cell 7 t

## 📤 Push Dataset to Opik

In [7]:
# Cell 7 - Push Dataset to Opik

import time  # Required for retry delays

def push_to_opik(opik_data, dataset_name="gretel-qa-dataset", max_retries=3):
    """Enhanced push with retry logic and better error handling"""
    
    if not opik_data:
        return False, "No data to push"
    
    print(f"📤 Pushing {len(opik_data)} items to Opik...")
    
    for attempt in range(max_retries):
        try:
            # Import and initialize Opik client
            import opik
            opik_client = opik.Opik()
            
            # Create or get dataset with enhanced description
            description = (
                f"Synthetic Q&A dataset generated using Gretel Navigator ({working_model}). "
                f"Contains {len(opik_data)} items with questions, contexts, and expected answers. "
                f"Generated on {time.strftime('%Y-%m-%d %H:%M:%S')}."
            )
            
            print(f"📊 Creating/finding dataset: {dataset_name}")
            opik_dataset = opik_client.get_or_create_dataset(
                name=dataset_name,
                description=description
            )
            
            print(f"✅ Dataset ready: {opik_dataset.name}")
            print(f"🆔 Dataset ID: {opik_dataset.id}")
            
            # Insert data with progress indication
            print("📥 Inserting data into Opik...")
            opik_dataset.insert(opik_data)
            
            # Brief wait for data propagation
            print("⏳ Waiting for data propagation...")
            time.sleep(3)
            
            print(f"✅ Successfully pushed {len(opik_data)} items!")
            print(f"📊 Dataset name: {opik_dataset.name}")
            print(f"🆔 Dataset ID: {opik_dataset.id}")
            
            # Show detailed sample of what was pushed
            if opik_data:
                sample = opik_data[0]
                print(f"\n📋 Sample item pushed:")
                print(f"   📝 Question: {sample['input'].get('question', 'N/A')[:80]}...")
                print(f"   💡 Answer: {sample['expected_output'][:80]}...")
                print(f"   📊 Metadata keys: {list(sample['metadata'].keys())}")
                
                # Show statistics
                total_questions = len(opik_data)
                has_context = sum(1 for item in opik_data if 'context' in item['input'])
                avg_q_len = sum(len(item['input'].get('question', '')) for item in opik_data) / total_questions
                avg_a_len = sum(len(item['expected_output']) for item in opik_data) / total_questions
                
                print(f"\n📈 Dataset statistics:")
                print(f"   📝 Total questions: {total_questions}")
                print(f"   📄 Items with context: {has_context}/{total_questions} ({has_context/total_questions*100:.1f}%)")
                print(f"   📏 Average question length: {avg_q_len:.1f} characters")
                print(f"   📏 Average answer length: {avg_a_len:.1f} characters")
            
            return True, opik_dataset.name
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ Attempt {attempt + 1}/{max_retries} failed: {error_msg[:150]}...")
            
            # Provide specific error guidance
            if "authentication" in error_msg.lower():
                print("🔐 Authentication issue - try running: opik.configure()")
            elif "network" in error_msg.lower() or "connection" in error_msg.lower():
                print("🌐 Network issue - check your internet connection")
            elif "quota" in error_msg.lower() or "limit" in error_msg.lower():
                print("📊 Rate limit or quota issue - wait before retrying")
            
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 3  # Progressive backoff
                print(f"🔄 Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"❌ All {max_retries} attempts failed")
                print("\n🔧 Troubleshooting steps:")
                print("   1. Check Opik authentication: opik.configure()")
                print("   2. Verify internet connection")
                print("   3. Check Comet workspace access")
                print("   4. Try with a smaller dataset first")
                return False, error_msg

    return False, "Max retries exceeded"

# Execute the push operation
print("📤 Starting push to Opik...")

# Validate prerequisites
prerequisites_ok = True
missing_items = []

# Check if opik_formatted_data exists
if 'opik_formatted_data' not in locals() and 'opik_formatted_data' not in globals():
    missing_items.append("opik_formatted_data")
    prerequisites_ok = False
else:
    # Get from appropriate scope
    if 'opik_formatted_data' in locals():
        data_to_push = opik_formatted_data
    else:
        data_to_push = globals()['opik_formatted_data']
    
    if not data_to_push:
        missing_items.append("valid opik_formatted_data")
        prerequisites_ok = False

# Check if working_model exists
if 'working_model' not in locals() and 'working_model' not in globals():
    missing_items.append("working_model")
    prerequisites_ok = False

if not prerequisites_ok:
    print(f"❌ Missing prerequisites: {missing_items}")
    print("📋 Please ensure these previous steps completed successfully:")
    print("   1. Cell 3: Find working model")
    print("   2. Cell 5: Data generation") 
    print("   3. Cell 6: Format conversion")
    
    # Set failure states
    success = False
    result = None
    
else:
    # Execute the push
    try:
        # Get the data from the appropriate scope
        if 'opik_formatted_data' in locals():
            data_to_push = opik_formatted_data
        else:
            data_to_push = globals()['opik_formatted_data']
        
        # Get working model
        if 'working_model' in locals():
            model_name = working_model
        else:
            model_name = globals()['working_model']
        
        print(f"📊 Data to push: {len(data_to_push)} items")
        print(f"🤖 Model used: {model_name}")
        
        # Push to Opik
        success, result = push_to_opik(data_to_push, "gretel-ai-qa-cookbook")
        
        if success:
            print(f"\n🎉 Integration completed successfully!")
            print(f"📊 Dataset '{result}' is now available in Opik")
            print(f"\n🔗 Next steps:")
            print(f"   1. Go to your Comet workspace")
            print(f"   2. Navigate to Opik → Datasets")
            print(f"   3. Find your dataset: {result}")
            print(f"   4. Use it in model evaluations!")
            print(f"   5. Run Cell 8 to verify the dataset")
            
        else:
            print(f"\n❌ Failed to complete integration")
            print(f"💡 Error details: {result}")
            print(f"\n🔧 Try these solutions:")
            print(f"   1. Check your Opik authentication")
            print(f"   2. Verify network connectivity")
            print(f"   3. Try with a smaller dataset")
            print(f"   4. Contact support if issue persists")
            
    except Exception as e:
        print(f"❌ Unexpected error during push: {e}")
        success = False
        result = str(e)

# Store results in global scope for verification step
globals()['success'] = success
globals()['result'] = result
if 'data_to_push' in locals():
    globals()['opik_formatted_data'] = data_to_push

# Summary
print(f"\n🎯 Push operation completed!")
print(f"   Success: {success}")
print(f"   Result: {result}")
print(f"   Next: Run Cell 8 to verify the dataset")

# Debug information
print(f"\n🔍 Variables available for verification:")
for var_name in ['success', 'result', 'opik_formatted_data']:
    if var_name in globals():
        val = globals()[var_name]
        if var_name == 'opik_formatted_data':
            print(f"   ✅ {var_name}: {len(val) if val else 0} items")
        else:
            print(f"   ✅ {var_name}: {val}")
    else:
        print(f"   ❌ {var_name}: Not available")

📤 Starting push to Opik...
📊 Data to push: 10 items
🤖 Model used: gretelai/auto
📤 Pushing 10 items to Opik...
📊 Creating/finding dataset: gretel-ai-qa-cookbook


OPIK: Created a "gretel-ai-qa-cookbook" dataset at https://www.comet.com/opik/api/v1/session/redirect/datasets/?dataset_id=0197b141-e92a-7bbe-9a1c-6e2b909a3536&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


✅ Dataset ready: gretel-ai-qa-cookbook
🆔 Dataset ID: 0197b141-e92a-7bbe-9a1c-6e2b909a3536
📥 Inserting data into Opik...
⏳ Waiting for data propagation...
✅ Successfully pushed 10 items!
📊 Dataset name: gretel-ai-qa-cookbook
🆔 Dataset ID: 0197b141-e92a-7bbe-9a1c-6e2b909a3536

📋 Sample item pushed:
   📝 Question: What is the primary function of machine learning in AI?...
   💡 Answer: Machine learning allows systems to learn from data without explicit programming,...
   📊 Metadata keys: ['source', 'generated', 'row_index', 'model', 'question_length', 'answer_length', 'has_context']

📈 Dataset statistics:
   📝 Total questions: 10
   📄 Items with context: 10/10 (100.0%)
   📏 Average question length: 64.9 characters
   📏 Average answer length: 140.1 characters

🎉 Integration completed successfully!
📊 Dataset 'gretel-ai-qa-cookbook' is now available in Opik

🔗 Next steps:
   1. Go to your Comet workspace
   2. Navigate to Opik → Datasets
   3. Find your dataset: gretel-ai-qa-cookbook
   4. Us

The gretel-qa-dataset dataset can now be viewed in the UI:

![gretel-qa-dataset](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/gretel_opik_integration_cookbook.png)

## ✅ Verify Dataset Creation

In [8]:
# Cell 8 - Enhanced Dataset Verification

def verify_opik_dataset(dataset_name):
    """Enhanced dataset verification with detailed information and error handling"""
    
    try:
        print(f"🔍 Verifying dataset: {dataset_name}")
        
        # Import opik with error handling
        try:
            import opik
            opik_client = opik.Opik()
        except ImportError:
            print("❌ Opik not installed. Run: pip install opik")
            return False
        except Exception as e:
            print(f"❌ Failed to initialize Opik client: {e}")
            print("💡 Try running: opik.configure() first")
            return False
        
        # Get the dataset with retry logic
        max_retries = 3
        dataset = None
        
        for attempt in range(max_retries):
            try:
                dataset = opik_client.get_dataset(dataset_name)
                break
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"⚠️ Attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
                else:
                    raise e
        
        if dataset is None:
            print(f"❌ Dataset '{dataset_name}' not found")
            return False
        
        print(f"✅ Dataset verified: {dataset.name}")
        print(f"🆔 Dataset ID: {dataset.id}")
        
        # Try to get additional dataset information
        try:
            # Get dataset items to show count and sample
            print(f"📊 Fetching dataset details...")
            
            # Note: Actual item counting depends on Opik API capabilities
            print(f"📋 Dataset successfully created and accessible")
            
        except Exception as e:
            print(f"⚠️ Could not fetch dataset details: {e}")
        
        # Enhanced usage instructions
        print(f"\n📋 How to view your dataset:")
        print(f"   1. Go to https://www.comet.com")
        print(f"   2. Navigate to your workspace")
        print(f"   3. Click on 'Opik' in the left sidebar")
        print(f"   4. Go to 'Datasets' tab")
        print(f"   5. Look for dataset: {dataset_name}")
        
        print(f"\n🧪 How to use in evaluations:")
        
        # Basic usage example
        basic_example = f'''
# Basic Usage Example:
import opik

# Initialize client
opik_client = opik.Opik()
dataset = opik_client.get_dataset('{dataset_name}')

@opik.track
def my_qa_model(input_data):
    question = input_data.get('question', '')
    context = input_data.get('context', '')
    
    # Your model logic here (example with simple response)
    if context:
        response = f"Based on the context: {{context[:100]}}..., the answer to '{{question}}' is..."
    else:
        response = f"The answer to '{{question}}' is..."
    
    return response

# Run evaluation
evaluation = opik.evaluate(
    dataset=dataset,
    task=my_qa_model,
    experiment_name="gretel-synthetic-eval-{{int(time.time())}}"
)

print(f"Evaluation completed: {{evaluation}}")
'''
        
        # Advanced usage with metrics
        advanced_example = f'''
# Advanced Usage with Custom Metrics:
import opik
from opik.evaluation.metrics import Equals, Contains

opik_client = opik.Opik()
dataset = opik_client.get_dataset('{dataset_name}')

@opik.track
def advanced_qa_model(input_data):
    question = input_data.get('question', '')
    context = input_data.get('context', '')
    
    # Your advanced model logic here
    # Could integrate with OpenAI, Anthropic, or local models
    response = "Your model's detailed response here"
    
    return response

# Define custom metrics
def semantic_similarity_metric(output, expected_output):
    # Implement semantic similarity check
    # Could use sentence transformers, embeddings, etc.
    return {{"score": 0.85, "reason": "High semantic similarity"}}

# Run evaluation with metrics
evaluation = opik.evaluate(
    dataset=dataset,
    task=advanced_qa_model,
    scoring_metrics=[
        Equals(),  # Exact match
        Contains(),  # Substring match
        semantic_similarity_metric  # Custom metric
    ],
    experiment_name="gretel-advanced-eval-{{int(time.time())}}"
)
'''
        
        print("📝 Basic Usage:")
        print(basic_example)
        
        print("\n📊 Advanced Usage with Metrics:")
        print(advanced_example)
        
        print(f"\n🔗 Additional Resources:")
        print(f"   📖 Opik Documentation: https://opik.comet.com/")
        print(f"   🧪 Evaluation Examples: https://opik.comet.com/evaluation/")
        print(f"   🤖 Model Integration: https://opik.comet.com/integrations/")
        
        return True
        
    except Exception as e:
        print(f"❌ Could not verify dataset: {e}")
        
        # Provide troubleshooting guidance
        print(f"\n🔧 Troubleshooting:")
        print(f"   1. Check if dataset name is correct: '{dataset_name}'")
        print(f"   2. Verify Opik authentication: opik.configure()")
        print(f"   3. Check if previous push step completed successfully")
        print(f"   4. Try refreshing your Comet workspace")
        
        # Check if variables exist for debugging
        debug_info = {}
        for var_name in ['opik_formatted_data', 'success', 'result']:
            if var_name in globals():
                debug_info[var_name] = f"Available ({type(globals()[var_name]).__name__})"
            else:
                debug_info[var_name] = "Not available"
        
        print(f"\n🐛 Debug information:")
        for var, status in debug_info.items():
            print(f"   {var}: {status}")
        
        return False

def check_verification_prerequisites():
    """Check if all required variables exist for verification"""
    required_vars = {
        'success': 'success' in locals() or 'success' in globals(),
        'result': 'result' in locals() or 'result' in globals(),
        'opik_formatted_data': 'opik_formatted_data' in locals() or 'opik_formatted_data' in globals()
    }
    
    missing_vars = [var for var, exists in required_vars.items() if not exists]
    
    if missing_vars:
        print(f"⚠️ Missing variables for verification: {missing_vars}")
        print("🔄 This usually means previous steps didn't complete successfully")
        return False, missing_vars
    
    return True, []

# Main verification execution
print("🔍 Starting dataset verification...")

# Check prerequisites
prereq_check, missing = check_verification_prerequisites()

if not prereq_check:
    print(f"❌ Cannot verify dataset - missing prerequisites: {missing}")
    print("📋 Please ensure these previous steps completed successfully:")
    print("   1. Cell 5: Data generation")
    print("   2. Cell 6: Format conversion") 
    print("   3. Cell 7: Push to Opik")
else:
    # Try to get the dataset name from previous results
    dataset_name_to_verify = None
    
    # Check for result from push operation
    if 'result' in locals() and 'success' in locals() and success:
        dataset_name_to_verify = result
    elif 'result' in globals() and 'success' in globals() and globals()['success']:
        dataset_name_to_verify = globals()['result']
    else:
        # Fallback to default name
        dataset_name_to_verify = "gretel-ai-qa-cookbook"
        print(f"⚠️ Using default dataset name: {dataset_name_to_verify}")
    
    if dataset_name_to_verify:
        print(f"🎯 Verifying dataset: {dataset_name_to_verify}")
        verification_success = verify_opik_dataset(dataset_name_to_verify)
        
        if verification_success:
            print(f"\n🎉 Verification completed successfully!")
            print(f"📊 Dataset '{dataset_name_to_verify}' is ready for use!")
        else:
            print(f"\n❌ Verification failed for dataset '{dataset_name_to_verify}'")
    else:
        print("❌ No dataset name available for verification")
        print("🔄 Please run the previous cells to generate and push a dataset first")

🔍 Starting dataset verification...
🎯 Verifying dataset: gretel-ai-qa-cookbook
🔍 Verifying dataset: gretel-ai-qa-cookbook
✅ Dataset verified: gretel-ai-qa-cookbook
🆔 Dataset ID: 0197b141-e92a-7bbe-9a1c-6e2b909a3536
📊 Fetching dataset details...
📋 Dataset successfully created and accessible

📋 How to view your dataset:
   1. Go to https://www.comet.com
   2. Navigate to your workspace
   3. Click on 'Opik' in the left sidebar
   4. Go to 'Datasets' tab
   5. Look for dataset: gretel-ai-qa-cookbook

🧪 How to use in evaluations:
📝 Basic Usage:

# Basic Usage Example:
import opik

# Initialize client
opik_client = opik.Opik()
dataset = opik_client.get_dataset('gretel-ai-qa-cookbook')

@opik.track
def my_qa_model(input_data):
    question = input_data.get('question', '')
    context = input_data.get('context', '')
    
    # Your model logic here (example with simple response)
    if context:
        response = f"Based on the context: {context[:100]}..., the answer to '{question}' is..."
   

## 🔄 Alternative: Load from Gretel Export Files

If you have pre-existing Gretel datasets exported as files, you can also import them:

In [9]:
def load_gretel_export(file_path, format_type="csv"):
    """
    Load a Gretel dataset export from local file.
    Supports CSV, JSON, and JSONL formats.
    """
    try:
        if format_type.lower() == "csv":
            df = pd.read_csv(file_path)
        elif format_type.lower() == "json":
            df = pd.read_json(file_path)
        elif format_type.lower() == "jsonl":
            df = pd.read_json(file_path, lines=True)
        else:
            raise ValueError("Supported formats: csv, json, jsonl")
        
        print(f"✅ Loaded {len(df)} records from {file_path}")
        print(f"📊 Dataset shape: {df.shape}")
        print(f"📋 Columns: {list(df.columns)}")
        
        # Display sample data from Gretel
        print("\n📄 Sample data from Gretel:")
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_colwidth', 100)
        print(df.head(3))
        
        # Show data types
        print(f"\n📈 Data types:")
        print(df.dtypes)
        
        # Basic statistics
        print(f"\n📊 Basic statistics:")
        if 'question' in df.columns:
            print(f"  - Average question length: {df['question'].str.len().mean():.1f} characters")
        if 'answer' in df.columns or 'truth' in df.columns:
            answer_col = 'answer' if 'answer' in df.columns else 'truth'
            print(f"  - Average answer length: {df[answer_col].str.len().mean():.1f} characters")
        if 'topic' in df.columns:
            print(f"  - Unique topics: {df['topic'].nunique()}")
        if 'difficulty' in df.columns or 'user_profile' in df.columns:
            diff_col = 'difficulty' if 'difficulty' in df.columns else 'user_profile'
            print(f"  - Difficulty distribution: {dict(df[diff_col].value_counts())}")
            
        return df
    
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None

# Example usage:
# df_gretel = load_gretel_export("your_gretel_export.csv", "csv")
# df_gretel = load_gretel_export("your_gretel_export.jsonl", "jsonl")

# Then convert and push to Opik:
# opik_data = convert_gretel_to_opik_format(df_gretel, "gretel-export")
# success, result = push_to_opik(opik_data, "gretel-imported-dataset")

## 🎯 Complete Integration Summary

This cookbook provides a complete workflow for integrating Gretel AI datasets with Opik:

### ✅ **What We Accomplished:**
1. **Authentication Setup** - Both Gretel and Opik API configurations
2. **Model Discovery** - Automatic detection of working Gretel models
3. **Synthetic Data Generation** - Using Gretel Navigator for Q&A creation
4. **Format Conversion** - Transform Gretel output to Opik-compatible format
5. **Dataset Import** - Push datasets to Opik for evaluation use
6. **Verification** - Confirm successful import and provide usage guidance

### 🔧 **Key Features:**
- **Robust Error Handling**: Multiple fallback strategies
- **Automatic Column Detection**: Smart mapping of data fields
- **Flexible Input**: Supports both live generation and file imports
- **Production Ready**: Comprehensive validation and user guidance

### 📊 **Use Cases:**
- **Model Testing**: Create evaluation datasets for Q&A models
- **Benchmarking**: Generate consistent test sets across experiments
- **Agent Optimization**: Provide training data for Opik's Agent Optimizer
- **Continuous Evaluation**: Regular model performance monitoring

### 🚀 **Next Steps:**
1. Customize the `source_text` with your domain-specific content
2. Adjust generation parameters based on your needs
3. Use the imported dataset in Opik evaluations
4. Scale up for larger dataset generation

This integration enables seamless data flow from Gretel's synthetic data generation capabilities into Opik's model evaluation and optimization ecosystem! 🎉